<a href="https://colab.research.google.com/github/takyaC/Finance/blob/main/EstimateAndActual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yahoo_fin

In [2]:
pip install pandas

In [1]:
pip install requests_html

In [2]:
from yahoo_fin.stock_info import get_analysts_info, get_cash_flow, get_income_statement
import pandas

/usr/local/lib/python3.6/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [75]:
tickers = [
           # it
           "WORK", "AAPL", "MSFT", "TSLA", "AMZN", "GOOGL", "ORCL", "ZM",  
           # vaccine
           "MRNA", "EBS", "BNTX", 
           # clean
           "PLUG", "WKHS", 
           # airline
           "DAL", 
           # ipo
           "U", "RPRX", 
           # other
           "DOCU", "PTON", "CRWD",
           "RKT", "ZI", "ROOT", "SBLK", "IAC", "FUTU", "ROKU", "SQ", 
          #  value
           "BMY", "ANTM"
           ]

In [83]:
data_of_ticker = []

for ticker in tickers:
    try:
      # アナリスト予想（予想EPS、予想売上）
      ai = get_analysts_info(ticker)
      earnings_estimate = ai["Earnings Estimate"].iloc[[1], [1, 3, 4]]
      revenue_estimate = ai["Revenue Estimate"].iloc[[1], [1, 3, 4]]
      
      # 損益計算書（四半期売上）
      ist = get_income_statement(ticker, False)
      total_revenue = ist.loc[["totalRevenue"], ist.columns[0]]

      # 単位付与(100万/１０億)
      total_revenue_by_million = total_revenue.values / 1000 / 1000
      if total_revenue_by_million < 1000:
        converted_total_revenue = "".join(str(total_revenue_by_million))[1:-1] + "M"
      else:
        converted_total_revenue = "".join(str(total_revenue_by_million / 1000))[1:-1] + "B"
      
      # 損益計算書（年間売上）
      isty = get_income_statement(ticker)
      total_revenue_yearly = isty.loc[["totalRevenue"], isty.columns[0]]

      # 単位付与(100万/１０億)
      total_revenue_yearly_by_million = total_revenue_yearly.values / 1000 / 1000
      if total_revenue_yearly_by_million < 1000:
        converted_total_revenue_yearly_by_million = "".join(str(total_revenue_yearly_by_million))[1:-1] + "M"
      else:
        converted_total_revenue_yearly_by_million = "".join(str(total_revenue_yearly_by_million / 1000))[1:-1] + "B"

      # キャッシュフロー計算書（純利益、営業キャッシュフロー、リスク（純利益<営業キャッシュフローが正常)）
      cfy = get_cash_flow(ticker)
      net_income_yearly = cfy.loc[["netIncome"], cfy.columns[0]]
      operating_cash_flow_yearly = cfy.loc[["totalCashFromOperatingActivities"], cfy.columns[0]]
      is_risk_cleard  = "OK" if operating_cash_flow_yearly.values > net_income_yearly.values else "NG"

      # 営業キャッシュフローマージン (15%以上推奨)
      cash_flow_margin_yearly = (operating_cash_flow_yearly.values / total_revenue_yearly.values)

      # 設定
      data = []
      data.extend([ticker])
      data.extend(earnings_estimate.values[0])
      data.extend(revenue_estimate.values[0])
      data.extend([converted_total_revenue])
      data.extend([converted_total_revenue_yearly_by_million])
      data.extend([is_risk_cleard])
      data.extend(cash_flow_margin_yearly)
      data_of_ticker.append(data)

      print(data)
    except:
      data_of_ticker.append([ticker])
      

['WORK', -0.03, -0.08, -0.05, '239.41M', '886.96M', '1.14B', '234.498M', '630.422M', 'OK', -0.019651915700911453]
['AAPL', 1.4, 3.99, 4.36, '102.27B', '315.85B', '332.69B', '64.698B', '274.515B', 'OK', 0.2938782944465694]
['MSFT', 1.64, 6.76, 7.47, '40.19B', '158.28B', '175.53B', '37.154B', '143.015B', 'OK', 0.42425619690242283]
['TSLA', 0.92, 2.33, 3.95, '10.16B', '31.05B', '45.54B', '8.771B', '24.578B', 'OK', 0.09785173732606396]
['AMZN', 7.15, 34.9, 45.41, '119.51B', '379.98B', '449.57B', '96.145B', '280.522B', 'OK', 0.13729404467385803]
['GOOGL', 15.64, 51.56, 61.3, '53.07B', '178.86B', '216.8B', '46.173B', '161.857B', 'OK', 0.33684054443119543]
['ORCL', 1.11, 4.36, 4.68, '10.07B', '40.09B', '41.11B', '9.8B', '39.068B', 'OK', 0.33631104740452544]
['ZM', 0.8, 2.91, 2.99, '814.43M', '2.58B', '3.56B', '777.196M', '622.658M', 'OK', 0.2439412968274719]
['MRNA', -0.3, -1.51, 11.68, '293.24M', '529.56M', '8.86B', '157.91M', '60.209M', 'OK', -7.622913517912605]
['EBS', 2.83, 7.03, 8.88, '5

In [84]:
pandas.DataFrame(data_of_ticker, columns=[
        "銘柄",
        "今四半期予想　EPS", "今年度予想　EPS", "次年度予想　EPS",
        "今四半期予想　売上", "今年度予想　売上", "次年度予想　売上",
        "前四半期売上", "前年度売上",
        "リスク",
        "営業キャッシュフローマージン"])


,銘柄,今四半期予想 EPS,今年度予想 EPS,次年度予想 EPS,今四半期予想 売上,今年度予想 売上,次年度予想 売上,前四半期売上,前年度売上,リスク,営業キャッシュフローマージン
0,WORK,-0.03,-0.08,-0.05,239.41M,886.96M,1.14B,234.498M,630.422M,OK,-0.019652
1,AAPL,1.40,3.99,4.36,102.27B,315.85B,332.69B,64.698B,274.515B,OK,0.293878
2,MSFT,1.64,6.76,7.47,40.19B,158.28B,175.53B,37.154B,143.015B,OK,0.424256
3,TSLA,0.92,2.33,3.95,10.16B,31.05B,45.54B,8.771B,24.578B,OK,0.097852
4,AMZN,7.15,34.90,45.41,119.51B,379.98B,449.57B,96.145B,280.522B,OK,0.137294
5,GOOGL,15.64,51.56,61.30,53.07B,178.86B,216.8B,46.173B,161.857B,OK,0.336841
6,ORCL,1.11,4.36,4.68,10.07B,40.09B,41.11B,9.8B,39.068B,OK,0.336311
7,ZM,0.80,2.91,2.99,814.43M,2.58B,3.56B,777.196M,622.658M,OK,0.243941
8,MRNA,-0.30,-1.51,11.68,293.24M,529.56M,8.86B,157.91M,60.209M,OK,-7.622914
9,EBS,2.83,7.03,8.88,556.69M,1.53B,1.91B,385.2M,1.106B,OK,0.169982
